In [1]:
# ----------------------------------------------------------------------------
# Project     : mrs - multi-turn response selection
# Created By  : Eungis
# Team        : AI Engineering
# Created Date: 2023-11-30
# Updated Date: 2023-12-24
# Purpose     : Make data_loader for loading data
# version     : 0.0.1
# ---------------------------------------------------------------------------

In [2]:
import logging
import pandas as pd
from importlib import reload  # Not needed in Python 2

reload(logging)
logging.basicConfig(
    format="%(message)s",
    level=logging.DEBUG,
)

DATA_ROOT = "../data/"
logger = logging.getLogger()
data = pd.read_csv(DATA_ROOT + "smilestyle_dataset.tsv", sep="\t")

In [3]:
from typing import List

cols = data.columns.tolist()
logger.debug(f"Columns: {cols}")
# use formal conversational data
data = data[["formal"]]

data["group"] = data["formal"].isnull().cumsum()
n_sessions = data["group"].iat[-1] + 1
logger.debug(f"Number of groups: {n_sessions}")

# split data into sessions
sessions: List[List[str]] = []
groups = data.groupby("group", as_index=False, group_keys=False)

for i, group in groups:
    session = group.dropna()["formal"].tolist()
    sessions += [session]

assert n_sessions == len(sessions)

Columns: ['formal', 'informal', 'android', 'azae', 'chat', 'choding', 'emoticon', 'enfp', 'gentle', 'halbae', 'halmae', 'joongding', 'king', 'naruto', 'seonbi', 'sosim', 'translator']
Number of groups: 236


In [6]:
sessions[0]

['안녕하세요. 저는 고양이 6마리 키워요.',
 '고양이를 6마리나요? 키우는거 안 힘드세요?',
 '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
 '가장 나이가 많은 고양이가 어떻게 돼요?',
 '여섯 살입니다. 갈색 고양이에요.',
 '그럼 가장 어린 고양이가 어떻게 돼요?',
 '한 살입니다. 작년에 분양 받았어요.',
 '그럼 고양이들끼리 안 싸우나요?',
 '저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.']

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
logger.debug(f"Special tokens: {tokenizer.special_tokens_map}")
logger.debug(f"EOS token & SEP token: {tokenizer.eos_token} / {tokenizer.sep_token}")
special_tokens = {"sep_token": "<SEP>"}
tokenizer.add_special_tokens(special_tokens)

Starting new HTTPS connection (1): huggingface.co:443
https://huggingface.co:443 "HEAD /klue/roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
Special tokens: {'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
EOS token & SEP token: [SEP] / [SEP]


1

In [49]:
logger.info(
    f"""Special tokens map: {tokenizer.special_tokens_map}
{tokenizer.eos_token}: {tokenizer.eos_token_id}
{tokenizer.sep_token}: {tokenizer.sep_token_id}
{tokenizer.mask_token}: {tokenizer.mask_token_id}"""
)

Special tokens map: {'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '<SEP>', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
[SEP]: 2
<SEP>: 32000
[MASK]: 4


In [47]:
import random

session = sessions[0]
mask_ratio = 0.15
corrupt_tokens = []
output_tokens = []
for i, utt in enumerate(session):
    original_token = tokenizer.encode(utt, add_special_tokens=False)
    n_mask = int(len(original_token) * mask_ratio)
    mask_positions = random.sample([x for x in range(len(original_token))], n_mask)
    corrupt_token = []
    for pos in range(len(original_token)):
        if pos in mask_positions:
            corrupt_token.append(tokenizer.mask_token_id)
        else:
            corrupt_token.append(original_token[pos])
    if i == len(session) - 1:
        output_tokens.extend(original_token)
        corrupt_tokens.extend(corrupt_token)
    else:
        output_tokens.extend(original_token + [tokenizer.sep_token_id])
        corrupt_tokens.extend(corrupt_token + [tokenizer.sep_token_id])

logger.debug(tokenizer.decode(output_tokens))
logger.debug(tokenizer.decode(corrupt_tokens))

안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요? <SEP> 여섯 살입니다. 갈색 고양이에요. <SEP> 그럼 가장 어린 고양이가 어떻게 돼요? <SEP> 한 살입니다. 작년에 분양 받았어요. <SEP> 그럼 고양이들끼리 안 싸우나요? <SEP> 저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.
안녕하세요. 저는 고양이 6마리 [MASK]요. <SEP> 고양이 [MASK] 6마리나요? 키우는 [MASK] 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 [MASK]진 않 [MASK]요. <SEP> 가장 나이가 [MASK]은 고양이가 어떻게 돼요? <SEP> 여섯 살입니다. 갈색 고양이에 [MASK]. <SEP> [MASK] 가장 어린 고양이가 어떻게 돼요? <SEP> 한 살입니다 [MASK] 작년에 분양 받았어요. <SEP> 그럼 고양이들끼리 안 싸우나 [MASK]? <SEP> 저희 일곱은 다같이 한 가족입니다 [MASK] 싸우는 일은 [MASK]어요.


In [53]:
# construct short sessions
k = 4
short_sessions = []
for session in sessions:
    for i in range(len(session) - k + 1):
        short_sessions.append(session[i : i + k])
logger.debug(len(short_sessions))
logger.info(short_sessions[0])
logger.info(short_sessions[1])

2762
['안녕하세요. 저는 고양이 6마리 키워요.', '고양이를 6마리나요? 키우는거 안 힘드세요?', '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.', '가장 나이가 많은 고양이가 어떻게 돼요?']
['고양이를 6마리나요? 키우는거 안 힘드세요?', '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.', '가장 나이가 많은 고양이가 어떻게 돼요?', '여섯 살입니다. 갈색 고양이에요.']


In [54]:
# construct negative response candidates
import random

all_utts = set()
for session in sessions:
    for utt in session:
        all_utts.add(utt)
all_utts = list(all_utts)
logger.info(f"Number of negative samples: {len(all_utts)}")

Number of negative samples: 3430


In [60]:
session = short_sessions[0]
urc_tokens = []
context_utts = []

for i in range(len(session)):
    utt = session[i]
    original_token = tokenizer.encode(utt, add_special_tokens=False)
    if i == len(session) - 1:
        positive_tokens = urc_tokens + original_token
        while True:
            random_neg_response = random.choice(all_utts)
            if random_neg_response not in context_utts:
                break
        # random negative response
        random_neg_response_token = tokenizer.encode(
            random_neg_response, add_special_tokens=False
        )
        random_tokens = urc_tokens + random_neg_response_token

        # context negative response
        context_neg_response = random.choice(context_utts)
        context_neg_response_token = tokenizer.encode(
            context_neg_response, add_special_tokens=False
        )
        context_neg_tokens = urc_tokens + context_neg_response_token
    else:
        urc_tokens += original_token + [tokenizer.sep_token_id]
    context_utts.append(utt)

logger.debug(tokenizer.decode(positive_tokens))
logger.debug(tokenizer.decode(random_tokens))
logger.debug(tokenizer.decode(context_neg_tokens))

안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요?
안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 아니요, 그렇게는 절대로 살기 싫습니다.
안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 안녕하세요. 저는 고양이 6마리 키워요.


In [67]:
import torch
import random
import logging
from importlib import reload  # Not needed in Python 2
from typing import List
from torch.utils.data import Dataset
from transformers import AutoTokenizer

MODEL_NAME = "klue/roberta-base"
DATA_ROOT = "../data/"
DATA_PATH = DATA_ROOT + "smilestyle_dataset.tsv"


class PostDataset(Dataset):
    def __init__(self, data_path: str, ctx_len: int = 4):
        self.logger = self._set_logger()

        # set tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        special_tokens = {"sep_token": "<SEP>"}
        self.tokenizer.add_special_tokens(special_tokens)

        # construct sessions
        sessions = self._construct_sessions(data_path)

        # construct short sessions
        self.short_sessions = self._construct_short_sessions(sessions, ctx_len=ctx_len)

        # get all utterances
        self.all_utts = self._get_utterances(sessions)

    def __len__(self):
        return len(self.short_sessions)

    def __getitem__(self, idx):
        # Input data for MLM
        session = self.short_sessions[idx]
        mask_ratio = 0.15
        self.corrupt_tokens = []
        self.output_tokens = []
        for i, utt in enumerate(session):
            original_token = self.tokenizer.encode(utt, add_special_tokens=False)

            mask_num = int(len(original_token) * mask_ratio)
            mask_positions = random.sample(
                [x for x in range(len(original_token))], mask_num
            )
            corrupt_token = []
            for pos in range(len(original_token)):
                if pos in mask_positions:
                    corrupt_token.append(self.tokenizer.mask_token_id)
                else:
                    corrupt_token.append(original_token[pos])

            if i == len(session) - 1:
                self.output_tokens += original_token
                self.corrupt_tokens += corrupt_token
            else:
                self.output_tokens += original_token + [self.tokenizer.sep_token_id]
                self.corrupt_tokens += corrupt_token + [self.tokenizer.sep_token_id]

        # Label for loss
        self.corrupt_mask_positions = []
        for pos in range(len(self.corrupt_tokens)):
            if self.corrupt_tokens[pos] == self.tokenizer.mask_token_id:
                self.corrupt_mask_positions.append(pos)

        # URC
        urc_tokens = []
        context_utts = []
        for i in range(len(session)):
            utt = session[i]
            original_token = self.tokenizer.encode(utt, add_special_tokens=False)
            if i == len(session) - 1:
                urc_tokens += [self.tokenizer.eos_token_id]
                self.positive_tokens = (
                    [self.tokenizer.cls_token_id] + urc_tokens + original_token
                )
                while True:
                    random_neg_response = random.choice(self.all_utts)
                    if random_neg_response not in context_utts:
                        break
                random_neg_response_token = self.tokenizer.encode(
                    random_neg_response, add_special_tokens=False
                )
                self.random_tokens = (
                    [self.tokenizer.cls_token_id]
                    + urc_tokens
                    + random_neg_response_token
                )
                context_neg_response = random.choice(context_utts)
                context_neg_response_token = self.tokenizer.encode(
                    context_neg_response, add_special_tokens=False
                )
                self.context_neg_tokens = (
                    [self.tokenizer.cls_token_id]
                    + urc_tokens
                    + context_neg_response_token
                )
            else:
                urc_tokens += original_token + [self.tokenizer.sep_token_id]
            context_utts.append(utt)

        return (
            self.corrupt_tokens,
            self.output_tokens,
            self.corrupt_mask_positions,
            [self.positive_tokens, self.random_tokens, self.context_neg_tokens],
            [0, 1, 2],
        )

    def collate_fn(self, sessions):
        """
        input:
            data: [(session1), (session2), ... ]
        return:
            batch_corrupt_tokens: (B, L) padded
            batch_output_tokens: (B, L) padded
            batch_corrupt_mask_positions: list
            batch_urc_inputs: (B, L) padded
            batch_urc_labels: (B)
            batch_mlm_attentions
            batch_urc_attentions

        batch가 3
        MLM = 3개의 입력데이터 (입력데이터별로 길이가 다름)
        URC = 9개의 입력데이터 (context는 길이가 다름, response candidate도 길이가 다름)
        """
        (
            batch_corrupt_tokens,
            batch_output_tokens,
            batch_corrupt_mask_positions,
            batch_urc_inputs,
            batch_urc_labels,
        ) = ([], [], [], [], [])
        batch_mlm_attentions, batch_urc_attentions = [], []
        # MLM, URC 입력에 대해서 가장 긴 입력 길이를 찾기
        corrupt_max_len, urc_max_len = 0, 0
        for session in sessions:
            (
                corrupt_tokens,
                output_tokens,
                corrupt_mask_positions,
                urc_inputs,
                urc_labels,
            ) = session
            if len(corrupt_tokens) > corrupt_max_len:
                corrupt_max_len = len(corrupt_tokens)
            positive_tokens, random_tokens, context_neg_tokens = urc_inputs
            if (
                max([len(positive_tokens), len(random_tokens), len(context_neg_tokens)])
                > urc_max_len
            ):
                urc_max_len = max(
                    [len(positive_tokens), len(random_tokens), len(context_neg_tokens)]
                )

        ## padding 토큰을 추가하는 부분
        for session in sessions:
            (
                corrupt_tokens,
                output_tokens,
                corrupt_mask_positions,
                urc_inputs,
                urc_labels,
            ) = session
            """ mlm 입력 """
            batch_corrupt_tokens.append(
                corrupt_tokens
                + [
                    self.tokenizer.pad_token_id
                    for _ in range(corrupt_max_len - len(corrupt_tokens))
                ]
            )
            batch_mlm_attentions.append(
                [1 for _ in range(len(corrupt_tokens))]
                + [0 for _ in range(corrupt_max_len - len(corrupt_tokens))]
            )

            """ mlm 출력 """
            batch_output_tokens.append(
                output_tokens
                + [
                    self.tokenizer.pad_token_id
                    for _ in range(corrupt_max_len - len(corrupt_tokens))
                ]
            )

            """ mlm 레이블 """
            batch_corrupt_mask_positions.append(corrupt_mask_positions)

            """ urc 입력 """
            # positive_tokens, random_tokens, context_neg_tokens = urc_inputs
            for urc_input in urc_inputs:
                batch_urc_inputs.append(
                    urc_input
                    + [
                        self.tokenizer.pad_token_id
                        for _ in range(urc_max_len - len(urc_input))
                    ]
                )
                batch_urc_attentions.append(
                    [1 for _ in range(len(urc_input))]
                    + [0 for _ in range(urc_max_len - len(urc_input))]
                )

            """ urc 레이블 """
            batch_urc_labels += urc_labels
        return (
            torch.tensor(batch_corrupt_tokens),
            torch.tensor(batch_output_tokens),
            batch_corrupt_mask_positions,
            torch.tensor(batch_urc_inputs),
            torch.tensor(batch_urc_labels),
            torch.tensor(batch_mlm_attentions),
            torch.tensor(batch_urc_attentions),
        )

    def _set_logger(self):
        logging.basicConfig(
            format="%(message)s",
            level=logging.DEBUG,
        )
        logger = logging.getLogger()
        return logger

    def _construct_sessions(self, data_path: str) -> List[List[str]]:
        data = pd.read_csv(data_path, sep="\t")
        cols = data.columns.tolist()
        logger.debug(f"Columns: {cols}")

        # use formal conversational data
        data = data[["formal"]]
        data["group"] = data["formal"].isnull().cumsum()
        n_sessions = data["group"].iat[-1] + 1
        logger.debug(f"Number of groups: {n_sessions}")

        # split data into sessions
        sessions: List[List[str]] = []
        groups = data.groupby("group", as_index=False, group_keys=False)

        for i, group in groups:
            session = group.dropna()["formal"].tolist()
            sessions += [session]
        assert n_sessions == len(sessions)
        return sessions

    def _construct_short_sessions(self, sessions, ctx_len):
        short_sessions = []
        for session in sessions:
            for i in range(len(session) - ctx_len + 1):
                short_sessions.append(session[i : i + ctx_len])
        return short_sessions

    def _get_utterances(self, sessions):
        all_utts = set()
        for session in sessions:
            for utt in session:
                all_utts.add(utt)
        return list(all_utts)

In [68]:
from torch.utils.data import DataLoader

post_dataset = PostDataset(data_path=DATA_PATH, ctx_len=4)
post_dataloader = DataLoader(
    post_dataset, batch_size=2, shuffle=True, collate_fn=post_dataset.collate_fn
)

https://huggingface.co:443 "HEAD /klue/roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
Columns: ['formal', 'informal', 'android', 'azae', 'chat', 'choding', 'emoticon', 'enfp', 'gentle', 'halbae', 'halmae', 'joongding', 'king', 'naruto', 'seonbi', 'sosim', 'translator']
Number of groups: 236


In [71]:
# Test Dataset
(
    corrupt_tokens,
    output_tokens,
    corrupt_mask_positions,
    urc_inputs,
    urc_labels,
) = post_dataset[0]
print(post_dataset.tokenizer.decode(corrupt_tokens))
print(post_dataset.tokenizer.decode(output_tokens))
print(corrupt_mask_positions)
print("####")
print(post_dataset.tokenizer.decode(urc_inputs[0]))
print(post_dataset.tokenizer.decode(urc_inputs[1]))
print(post_dataset.tokenizer.decode(urc_inputs[2]))

안녕하세요. 저는 고양이 6 [MASK] 키워요. <SEP> [MASK]를 6마리 [MASK]요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 [MASK] [MASK] 않아요. <SEP> 가장 나이가 많은 고양이가 [MASK] 돼요?
안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요?
[8, 13, 17, 36, 37, 50]
####
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> [SEP] 가장 나이가 많은 고양이가 어떻게 돼요?
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> [SEP] 고양이가 사람 음식을 너무 많이 먹으면 배탈이 납니다.
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> [SEP] 안녕하세요. 저는 고양이 6마리 키워요.


In [81]:
# Test DataLoader
(
    batch_corrupt_tokens,
    batch_output_tokens,
    batch_corrupt_mask_positions,
    batch_urc_inputs,
    batch_urc_labels,
    batch_mlm_attentions,
    batch_urc_attentions,
) = next(iter(post_dataloader))

In [82]:
post_dataset.tokenizer.decode(batch_urc_inputs[0])

'[CLS] csr가 뭔지는 모르겠지만, 도어 대시라는 회사는 알고 있습니다. 큰 회사였잖아요. <SEP> csr는 고객 서비스 담당자의 약자입니다. 해당 부서가 축소되면서 저도 잘렸습니다. <SEP> 그러면 지금은 뭘 하는 중이세요? <SEP> [SEP] 경쟁 회사에 취직하고 나서는, 일 뿐만 아니라 취미활동도 하고 있습니다. 퇴근하고 나면 근처에 있는 종합체육시설에서 수영을 해요.'